In [1]:
import ee
import geemap
import time

In [2]:
Map=geemap.Map()

In [3]:
trainData = ee.FeatureCollection("projects/ee-epgalanis/assets/Final_data_9k"),
Region = ee.Geometry.Polygon([[[140.96126303942492,-33.99133319528273],
  [140.96126303942492,-38.110919172137265],
  [146.30061850817492,-39.02144634571554],
  [150.07991538317492,-37.52075013904677],
  [140.96126303942492,-33.99133319528273]]])
Sentinel2A = ee.ImageCollection("COPERNICUS/S2_SR")

LAND_CLASS = 0
WATER_CLASS = 1
BANDS = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']

ic = Sentinel2A \
      .filterDate('2019-08-01', '2019-12-01') \
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
      
ic = ic.filter(ee.Filter.notNull(ic.first().propertyNames())) 


Map.setCenter(143.96306, -37.74696, 12)

# # print (classifier.explain())

input_data = ic.median().select(BANDS)

split = 0.7;  # Roughly 70% training, 30% testing.
samples = ic.randomColumn('random')
training = samples.filter(ee.Filter.lt('random', split))
validation = samples.filter(ee.Filter.gte('random', split))

classifier = ee.Classifier.smileRandomForest(400).train(training, 'class', BANDS)

print('Training size:', training.size())
print('Test size:', validation.size())

classified = input_data.classify(classifier)
trainAccuracy = classifier.confusionMatrix()

print(' Training Confusion Matrix   ')
print('                 True         ')
print('                 Water   Land ')
#print('Predicted  Water ' + trainAccuracy.getInfo()[0][0] + '      ' + trainAccuracy.getInfo()[0][1])
#print('           Land  ' + trainAccuracy.getInfo()[1][0] + '      ' + trainAccuracy.getInfo()[1][1])

print('------------------------------')

Training size: ee.Number({
  "functionInvocationValue": {
    "functionName": "Collection.size",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.randomColumn",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.filter",
                            "arguments": {
                              "collection": {
                                "functionInvocationValue": {
                                  "functionName": "Collection.filter",
                                

In [4]:
validated = validation.classify(classifier)
# testAccuracy = validated.confusionMatrix()
testAccuracy = validated.errorMatrix('class', 'classification')

print('      Test Confusion Matrix   ')

print('                 True         ')
print('                 Water   Land ')
#print('Predicted  Water ' + testAccuracy.getInfo()[0][0] + '      ' + testAccuracy.getInfo()[0][1])
#print('           Land  ' + testAccuracy.getInfo()[1][0] + '      ' + testAccuracy.getInfo()[1][1])
print('------------------------------')

# Get a confusion matrix representing expected accuracy.
#print('Validation overall accuracy: ', testAccuracy.accuracy())

trees = ee.List(ee.Dictionary(classifier.explain()).get('trees'))
dummy = ee.Feature(Region)

col = ee.FeatureCollection(trees.map(lambda x: dummy.set('tree', x)))

      Test Confusion Matrix   
                 True         
                 Water   Land 
------------------------------


In [5]:
#task = ee.batch.Export.table.toAsset(col)#,'save_classifier','RandomForest01')
#
#task.start()

task = ee.batch.Export.table.toAsset(**{
    'collection':  col,
    'description':  'save_classifier',
    'assetId': 'projects/ee-dam-app/assets/model'
    
    #users/spry-optics-357411/exampleExport
    #projects/dam-ee-markblashki1-test/assets/locations
})
task.start()
#task.status()

while task.active():
    print(task.status())
    time.sleep(0.5)
    
print(task.status())
print('Success')

palette = [
  'A52A2A', # Land
  '0000FF', # Water
]

#Map.addLayer(classified.clip(Region), {'palette': palette, 'min': 0, 'max': 2}, 'classification CART')

EEException: Name "projects/earthengine-public/assets/ee-dam-app" is invalid. The portion of the Asset name following "projects/*/assets/" must have at least 3 segments and at most 10 segments.